# 자치구 공원분석

In [1]:
!apt-get install -y fonts-nanum > /dev/null
!fc-cache -fv > /dev/null
!rm -rf ~/.cache/matplotlib > /dev/null
# 런타임 다시시작

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['axes.unicode_minus'] = False
plt.rc('font', family='NanumBarunGothic')

In [9]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving 서울공원전처리 (1).csv to 서울공원전처리 (1).csv


# 1. 자치구별 공원 면적과 공원수

In [10]:
park = pd.read_csv(filename)
park.head(5)

,공원명,지역,경도,위도,면적,분류,크기
0,남산도시자연공원,중구,126.990377,37.550140,2896887,대형,15
1,길동생태공원,강동구,127.154779,37.540394,80683,소형,3
2,서울대공원,과천시,127.019846,37.426449,9132690,대형,15
3,서울숲,성동구,127.041798,37.543072,480994,중형,7
4,월드컵공원,마포구,126.878907,37.571805,2284085,대형,15


In [11]:
df = park.groupby('지역')[['면적']].agg(['sum', 'count'])
df

면적      
           sum count
지역                  
강남구    3626398     7
강동구    1526390     7
강북구    3714902     4
강서구    2194154     9
과천시    9132690     1
관악구   10431453     2
광진구     584002     2
구로구    1617657     4
금천구      57568     4
노원구   12024307     2
도봉구    1084218     6
동대문구    458664     4
동작구    3706739     7
마포구    2685112     5
서대문구   3321477     4
서초구   13359636     6
성동구     626839     5
성북구    1198327     3
송파구    2838993     7
양천구    1203428     5
영등포구    424446     5
용산구     247194     2
은평구    2986459     7
종로구    3907681    12
중구     2943304     4
중랑구    7078975     6

In [12]:
df.drop(index='과천시', inplace=True)
df.columns =['공원면적','공원수']
df.head(3)

,공원면적,공원수
지역,,
강남구,3626398,7
강동구,1526390,7
강북구,3714902,4


In [13]:
df.shape

(25, 2)

# 2.구별 공원 면적 비율 및 인당 공원 면적 비교

In [14]:
seoul_df = pd.read_csv('서울시 인구와 면적.txt', sep='\t')
seoul_df.head(3)

,기간,지역,인구,면적,인구밀도(명/㎢)
0,2020,합계,"9,911,088",605.23,"16,376"
1,2020,종로구,"158,996",23.91,"6,649"
2,2020,중구,"134,635",9.96,"13,517"


In [ ]:
# 1. 인덱를 없애고
seoul_df.drop(index=0, inplace=True)

In [ ]:
# 2. '지역'컬럼을 인덱스
seoul_df.set_index('지역', inplace=True)
seoul_df 

In [18]:
# 3. 인구와 면적 공원수 를 합침 merge join concat
df = df.join(seoul_df[['인구','면적']])
df.head(3)
# 컬럼은 [지역 공원면적 공원수 인구 면적]

,공원면적,공원수,인구,면적
지역,,,,
강남구,3626398,7,"544,055",39.50
강동구,1526390,7,"463,998",24.59
강북구,3714902,4,"311,569",23.60


In [19]:
# 4. 면적 * 1000000
df['면적'] = df.면적 * 1000000
df['인구'] = df.인구.apply(lambda x : int(x.replace(',', '')))

,공원면적,공원수,인구,면적
지역,,,,
강남구,3626398,7,544055,39500000.0
강동구,1526390,7,463998,24590000.0
강북구,3714902,4,311569,23600000.0


In [21]:
# 5. 면적비율, 인당면적
df['면적비율'] = df.공원면적 / df.면적 * 100
df['인당면적'] = df.공원면적 / df.인구
df.head(3)

,공원면적,공원수,인구,면적,면적비율,인당면적
지역,,,,,,
강남구,3626398,7,544055,39500000.0,9.180754,6.665499
강동구,1526390,7,463998,24590000.0,6.207361,3.289648
강북구,3714902,4,311569,23600000.0,15.741110,11.923208


In [ ]:
#6. 시각화 barh - 자치구별 공원면적 비율
df.면적비율.sort_values().plot(kind='barh', grid=True, figsize=(10,6))
plt.title('자치구별 공원면적비율', size=15)
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
order = df.면적비율.sort_values(ascending=False).index
sns.barplot(x='면적비율', y=df.index, data=df, order=order)
plt.grid()
plt.title('자치구별 공원면적비율', size=15)
plt.show()

In [ ]:
#7. 시각화 barh - 인당공원 면적 높은순으로
df.인당면적.sort_values().plot(kind='barh', grid=True, figsize=(10,6))
plt.title('자치구별 인당 공원면적비율', size=15)
plt.show()
#8. 저장

In [ ]:
plt.figure(figsize=(12,8))
order = df.인당면적.sort_values(ascending=False).index
sns.barplot(x='인당면적', y=df.index, data=df, order=order)
plt.grid()
plt.title('자치구별 인당 공원면적비율', size=15)
plt.show()

In [27]:
df.to_csv('자치구별공원현황.csv') # 주의 index=False 안됨